In [52]:
from perceval.backends.core.github import GitHub
from datetime import date
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import os
import sqlalchemy as s
import augur
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize 
import nltk.data
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import sentiment
from nltk import word_tokenize

In [53]:
if("notebooks" not in os.getcwd()):
    os.chdir("notebooks")
augurApp = augur.Application('../augur.config.json')
connect = augurApp.ghtorrentplus()
DB_STR = 'mysql+pymysql://{}:{}@{}:{}/{}?charset=utf8'.format(
            '', '', '',\
            '', ''
        )
db = s.create_engine(DB_STR)

In [54]:
table_names = s.inspect(db).get_table_names()
print(table_names)

['github_issues', 'issue_response_time', 'mail_lists', 'mail_lists_sentiment_scores', 'mailing_list_jobs']


In [55]:
if("github_issues" in table_names):
    lists_createdSQL = s.sql.text("""SELECT repo FROM github_issues""")
    df1 = pd.read_sql(lists_createdSQL, db)
    #print(df1)
    val = True

In [56]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
sid = SentimentIntensityAnalyzer()
col = 'score','sentiment'

In [57]:
SQL = """SELECT * FROM github_issues"""
df_users =  pd.read_sql(SQL, db)
#print(df_users)

In [59]:
grouped = df_users.groupby('subject').groups
num = 0
for group in grouped:
    print(group)
    issue_num = (df_users.loc[df_users['subject'] == group]['issue_number']).values
    messages = (df_users.loc[df_users['subject'] == group]['body']).tolist()
    print("Issue number: ",issue_num[0])
    num+=1
    #break
    for i in messages:
        #print(i)
        sentences = tokenizer.tokenize(i)
        compound = 0
        part = 0
        for sentence in sentences:
            scores = sid.polarity_scores(sentence)
            compound+=scores['compound']
            part+=1
            #print(sentence,scores)
            #if(scores['compound'] < 0.1):
            #    print(sentence,scores['compound'])
        avg = compound/part
        if(avg < -0.5):
            print(i,avg)
            print("\n\n\n\n\n\n")
    #if(num == 40):
    #    break
    num+=1
    print("\n\n\n")
print(num)

A new New CTC Meeting Schedule Proposal
Issue number:  29




AST, IR, bytecode and friends - standards and inovation
Issue number:  104
@Qard is this a lack of hooks problem, or are people actually modifying how non-exported code runs? -0.6514











Add @bengl to @nodejs/security
Issue number:  149




Adding @gibfahn to help repo / perhaps give collaborators group access
Issue number:  58




Allow passing objects to `util.promisify`
Issue number:  174




Async iterators
Issue number:  53




Breaking Changes
Issue number:  13




Buffer constructor: commit to deprecating
Issue number:  87




Buffer constructor: opt-in deprecation flag?
Issue number:  88




Buffer constructor: random-fill in Node.js 8.0.0?
Issue number:  90




Buffer constructor: should we backport zero-filling?
Issue number:  91




Buffer constructor: zero-fill in Node.js 8.0.0?
Issue number:  89




CITGM abuse
Issue number:  164
I totally agree with @MylesBorins:

> Citgm results are analyzed, and when a

Issue number:  16




Scheduling Meetings
Issue number:  14




Strict HTTP spec compliance
Issue number:  5




Suggested change for Stand Up
Issue number:  119




Summit / Dinner with TC39 Monday May 22nd
Issue number:  67




Support awaiting values through `util.awaitable`
Issue number:  12
@misterdjules there's no way to determine how a promise was rejected; a throw is indistinguishable from an explicit rejection of the same value. -0.6486











TC-39 Discussion: Accessing module metadata
Issue number:  133




TC-39 Discussion: Modules with failed evaluation
Issue number:  132




TC-39 Discussion: Seeking input on import options
Issue number:  131




Tail call optimiztion (TCO) in ES6 & Node.js
Issue number:  3
Implicit TCO seems really bad to me if it breaks expectations of debugability without specifically opting in...
 -0.5849











The future of GYP in Node.js
Issue number:  2




Thinking Point: Will requiring 2 Collaborators' POV improve reviews?
Issue number: 